In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import time
# import gensim

2023-05-06 18:28:29.262463: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 18:28:29.339866: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 18:28:29.341140: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 18:28:30.637931: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
data = pd.read_csv("./dataset/inspiration.csv", index_col = 0)
new_data = data[['Category', 'Quote']]
# new_data

In [3]:
new_data['Category'].unique()

array(['LOVE', 'LISTENING', 'STEWARDSHIP', 'RESILIENCE', 'EXPLORING',
       'KINDNESS', 'HELPING OTHERS', 'OPTIMISM', 'GIVING',
       'COMMON GROUND', 'HOPE', 'ACHIEVEMENT', 'GRATITUDE', 'CHARITY',
       'PRACTICE', 'GET OUTSIDE', 'FORGIVENESS', 'CHARACTER', 'LAUGHTER',
       'HUMILITY', 'ACCEPTANCE', 'DEDICATION', 'BELIEVE IN YOURSELF',
       'SMILE', 'PEACE', 'PERSPECTIVE', 'LIVE YOUR DREAMS', 'CONFIDENCE',
       'INNOVATION', 'LEARNING', 'UNITY', 'DISCOVERY', 'SELFLESSNESS',
       'FITNESS', 'LEADERSHIP', 'FRIENDSHIP', 'CURIOSITY',
       'ENCOURAGEMENT', 'SPREAD YOUR WINGS', 'HARD WORK', 'LIVE LIFE',
       'JOY', 'WISDOM', 'PERSISTENCE', 'INSPIRATION', 'OVERCOMING',
       'CREATIVITY', 'FAMILY', 'CHEER', 'GIVING BACK', 'COMPASSION',
       'AMBITION', 'LOYALTY', 'GRIT', 'EDUCATION', 'APPRECIATING NATURE',
       'INTEGRITY', 'EMPATHY', 'SELF-CARE', 'TEAMWORK',
       'INCLUDING OTHERS', 'COURAGE', 'PERSEVERANCE', 'APPRECIATION',
       'MOTIVATION', 'MENTORING', 'HUMOR', '

In [4]:
file1 = open("MyFile.txt", "w")
the_length = new_data.shape[0]
for i in range(0,the_length):
    file1.write(new_data["Category"][i])
    file1.write(": ")
    file1.write(new_data["Quote"][i])
    file1.write("\n")
    # file1.write("\n")
file1.close()

In [5]:
from keras.callbacks import LambdaCallback
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file
import string

print('\nPreparing the sentences...')
max_sentence_len = 40
with open("MyFile.txt") as file_:
  docs = file_.readlines()
sentences = [[word for word in doc.lower().translate(string.punctuation).split()] for doc in docs]
# print(sentences)
print('Num sentences:', len(sentences))


Preparing the sentences...
Num sentences: 1797


In [6]:
text = open("MyFile.txt", "r").read()
maxlen = 60 #extract sequences of length 60
step = 3
sentences = []	#holds extracted sequences
next_chars = [] #holds the targets
for i in range(0, len(text)-maxlen, step):
	sentences.append(text[i:i+maxlen])
	next_chars.append(text[i+maxlen])
#VECTORIZATION
chars = sorted(set(text))
char_indices = dict((char, chars.index(char)) for char in chars)
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
	for t, char in enumerate(sentence):
		x[i, t, char_indices[char]] = 1
	y[i, char_indices[next_chars[i]]] = 1

/tmp/ipykernel_10140/2596606510.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
/tmp/ipykernel_10140/2596606510.py:13: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [7]:
from tensorflow.keras import layers
model = tf.keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation="softmax"))

2023-05-06 18:28:34.745146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-06 18:28:34.745811: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-05-06 18:28:35.376523: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/conca

In [20]:
model.compile(loss="categorical_crossentropy", optimizer="adam", run_eagerly=True)

In [13]:
def sample(preds, temperature=1.0):
	preds = np.asarray(preds).astype('float64')
	preds = np.log(preds) / temperature
	exp_preds = np.exp(preds)
	preds = exp_preds / np.sum(exp_preds)
	probas = np.random.multinomial(1, preds, 1)
	return np.argmax(probas)

In [21]:
import random
for epoch in range(1, 60):
    model.fit(x, y, batch_size=512)
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = x[start_index: start_index + maxlen]
    for i in range(400): #generates 400 length string
        preds = model.predict(generated_text)[0]
        next_index = sample(preds, temperature=1.0)
        next_char = chars[next_index]
        generated_text += next_char
        generated_text = generated_text[1:]

141/141 [==============================] - 111s 783ms/step - loss: 2.2953


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.